In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from copy import deepcopy
import pytesseract as tess

In [4]:
def preprocess(img):
    b, g, r = cv2.split(img)
    img = cv2.merge([r, g, b])
    plt.imshow(img)
    
    r, g, b = cv2.split(img)
    img = cv2.merge([b, g, r])
    img_blurred = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(img_blurred, cv2.COLOR_BGR2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
    
    ret2, threshold_img = cv2.threshold(sobelx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return threshold_img

In [5]:
def clean_plate(plate):
    print("cleaning number plate")
    
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)[1]
    im1, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        
        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        
        x, y, w, h = cv2.boundingRect(max_cnt)
        
        if not radioCheck(max_cntArea, w, h):
            return plate, None
        cleaned_final = thresh[y:y + h, x: x + w]
        
        return cleaned_final, [x, y, w, h]
    
    else:
        return plate, None    

In [6]:
def extract_contours(threshold_img):
    element = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize = (17, 3))
    morph_img_threshold = threshold_img.copy()
    cv2.morphologyEx(src = threshold_img, op = cv2.MORPH_CLOSE, kernel = element, dst = morph_img_threshold)
    cv2.imshow("Morphed", morph_img_threshold)
    cv2.waitKey(0)
    
    im2, contours, hierarchy = cv2.findContours(morph_img_threshold, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    return contours

In [7]:
def radioCheck(area, width, height):
    ratio = float(width) / float(height)
    
    if ratio < 1:
        ratio = 1 / ratio
        
    aspect = 4.7272
    min = 15 * aspect * 15
    max = 125 * aspect * 125
    
    rmin = 3
    rmax = 6
    
    if area in range(min, max + 1) or ratio in range(rmin, rmax + 1):
        return True

    return False
        

In [8]:
def isMaxWhite(plate):
    avg = np.mean(plate)
    
    if avg >= 115:
        return True
    
    return False

In [9]:
def validateRotationAndRatio(rect):
    (x, y), (width, height), rect_angle = rect
    
    if width > height:
        angle = -rect_angle
    else:
        angle = 90 + rect_angle
    
    if angle > 15:
        return False
    
    if height == 0 or width == 0:
        return False
    
    area = height*width
    
    if not radioCheck(area, width, height):
        return False

    return True

In [10]:
def cleanAndRead(img, contours):
    for i, cnt in enumerate(contours):
        min_rect = cv2.minAreaRect(cnt)
        
        if validateRotationAndRatio(min_rect):
            
            x, y, w, h = cv2.boundingRect(cnt)
            plate_img = img[y: y + h, x: x + w]
            
            if (isMaxWhite(plate_img)):
                clean_plate, rect = cleanPlate(plate_img)
                
            if rect:
                x1, y1, w1, h1 = rect
                x, y, w, h = x + x1, y + y1, w, h
                b, g, r = cv2.split(clean_plate)
                clean_plate = cv2.merge([r, g, b])
                plt.imshow(clean_plate)
                r, g, b = cv2.split(clean_plate)
                clean_plate = cv2.merge([b, g, r])
                
                cv2.waitKey(0)
                plate_im = Image.fromarray(clean_array)
                
                text = tess.image_to_string(plate_im, lang = "eng")
                print("Detected Text: ", text)
                
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                b, g, r = cv2.split(img)
                img = cv2.merge([r, g, b])
                plt.imshow(img)
                r, g, b = cv2.split(img)
                img = cv2.merge([b, g, r])
                
                cv2.waitKey(0)

In [ ]:
if __name__ == "__main__":
    print("Detecting Plate")
    
    img = cv2.imread("car.jpg")
    
    threshold_img = preprocess(img)
    contours = extract_contours(threshold_img)
    
    cleanAndRead(img, contours)

Detecting Plate
